In [ ]:
# this is a test about recomputation acceleration for KV cache

In [ ]:
import torch

In [ ]:
# input parameters
model_dim = 2000
seq_len = 1000
batch_size = 1

In [ ]:
# generate W_q, W_k, W_v

In [ ]:
W_q = torch.randn(model_dim, model_dim).half()
W_k = torch.randn(model_dim, model_dim).half()
W_v = torch.randn(model_dim, model_dim).half()
prefil_kcache = torch.randn(batch_size,seq_len, model_dim).half()
prefil_vcache = torch.randn(batch_size,seq_len, model_dim).half()
new_token = torch.randn(batch_size,1, model_dim).half()
all_tokens = torch.randn(batch_size,seq_len, model_dim).half()

In [ ]:
W_q = W_q.to("cuda")
W_k = W_k.to("cuda")
W_v = W_v.to("cuda")

In [ ]:
# all compute no kv cache
import time
all_tokens = all_tokens.to("cuda")
start = time.time()

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # You can specify the model size, e.g., "gpt2-medium" for a different model size
model = GPT2LMHeadModel.from_pretrained(model_name, use_cache=True) # disable cache
tokenizer = GPT2Tokenizer.from_pretrained(model_name) 

# Define a prompt
prompt = "Once upon a time, in a land far, far away,"

# Generate a sentence based on the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

# Decode and print the generated sentence
generated_sentence = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Sentence: ", generated_sentence)

/usr/scratch/haokang/anaconda3/envs/kh/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


before torch.Size([1, 13, 768]) torch.Size([1, 13, 768]) torch.Size([1, 13, 768])
after torch.Size([1, 12, 13, 64]) torch.Size([1, 12, 13, 64]) torch.Size([1, 12, 13, 64])
attn_output torch.Size([1, 12, 13, 64]) attn_weights torch.Size([1, 12, 13, 13])
before torch.Size([1, 1, 768]) torch.Size([1, 1, 768]) torch.Size([1, 1, 768])
after torch.Size([1, 12, 1, 64]) torch.Size([1, 12, 14, 64]) torch.Size([1, 12, 14, 64])
attn_output torch.Size([1, 12, 1, 64]) attn_weights torch.Size([1, 12, 1, 14])
before torch.Size([1, 1, 768]) torch.Size([1, 1, 768]) torch.Size([1, 1, 768])
after torch.Size([1, 12, 1, 64]) torch.Size([1, 12, 15, 64]) torch.Size([1, 12, 15, 64])
attn_output torch.Size([1, 12, 1, 64]) attn_weights torch.Size([1, 12, 1, 15])
before torch.Size([1, 1, 768]) torch.Size([1, 1, 768]) torch.Size([1, 1, 768])
after torch.Size([1, 12, 1, 64]) torch.Size([1, 12, 16, 64]) torch.Size([1, 12, 16, 64])
attn_output torch.Size([1, 12, 1, 64]) attn_weights torch.Size([1, 12, 1, 16])
before

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # You can specify the model size, e.g., "gpt2-medium" for a different model size
model = GPT2LMHeadModel.from_pretrained(model_name, use_cache=False) # disable cache
tokenizer = GPT2Tokenizer.from_pretrained(model_name) 

# Define a prompt
prompt = "Once upon a time, in a land far, far away,"

# Generate a sentence based on the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

# Decode and print the generated sentence
generated_sentence = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Sentence: ", generated_sentence)

/usr/scratch/haokang/anaconda3/envs/kh/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


before torch.Size([1, 13, 768]) torch.Size([1, 13, 768]) torch.Size([1, 13, 768])
after torch.Size([1, 12, 13, 64]) torch.Size([1, 12, 13, 64]) torch.Size([1, 12, 13, 64])
attn_output torch.Size([1, 12, 13, 64]) attn_weights torch.Size([1, 12, 13, 13])
before torch.Size([1, 14, 768]) torch.Size([1, 14, 768]) torch.Size([1, 14, 768])
after torch.Size([1, 12, 14, 64]) torch.Size([1, 12, 14, 64]) torch.Size([1, 12, 14, 64])
attn_output torch.Size([1, 12, 14, 64]) attn_weights torch.Size([1, 12, 14, 14])
before torch.Size([1, 15, 768]) torch.Size([1, 15, 768]) torch.Size([1, 15, 768])
after torch.Size([1, 12, 15, 64]) torch.Size([1, 12, 15, 64]) torch.Size([1, 12, 15, 64])
attn_output torch.Size([1, 12, 15, 64]) attn_weights torch.Size([1, 12, 15, 15])
before torch.Size([1, 16, 768]) torch.Size([1, 16, 768]) torch.Size([1, 16, 768])
after torch.Size([1, 12, 16, 64]) torch.Size([1, 12, 16, 64]) torch.Size([1, 12, 16, 64])
attn_output torch.Size([1, 12, 16, 64]) attn_weights torch.Size([1, 1